In [62]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import NoTranscriptFound, TranscriptsDisabled, VideoUnavailable
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown
from openai import OpenAI
from pytube import YouTube


In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key found. Please enter the API key properly")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found but it doesn't start with sk-proj-; Please check whether you are using the right API key")
elif api_key.strip() != api_key:
    print("An API key was found and it looks good so far!")

In [ ]:
def get_transcript(videoId):
    try:
        transcripts = YouTubeTranscriptApi.list_transcripts(videoId)
        try:
            manual = transcripts.find_manually_created_transcript(["en"])
            print("Manually created transcript found")
            return manual.fetch()
        except NoTranscriptFound:
            print("Not found- Manually created transcipt")
        try: 
            auto = transcripts.find_generated_transcript(["en"])
            print("Auto generated transcript found")
            return auto.fetch()
        except NoTranscriptFound:
            print("Not found - Auto generated Transcript")
        return None
    except TranscriptsDisabled:
        print("Transcripts are disable for this video!")
        return None
    except VideoUnavailable:
        print("Video is unavailable or private")
        return None
    except Exception as e:
        print("Unexpected error")
        return None

In [ ]:
videoId = "GGJ5k8U5ccM"
transcript = get_transcript(videoId)
full_text = " "
if transcript:
    full_text = " ".join([t.text for t in transcript.snippets])
print(full_text)

Not found- Manually created transcipt
Auto generated transcript found
foreign [Music] foreign [Music] ERS today we will be discussing the problem of the day which is jumping numbers so I found this problem absolutely very interesting so let us just quickly start hi vedant hi yes hi Shiva CR Lee fgmg gives me a present okay fair enough so let's start with this problem so I think you have already read about this problem now in this problem you are asked so let us first understand that you are given a value X and you need to find a value which is a magical number okay now the question is that you need to find the nearest magical number to x nearest means less than equal okay so now the question is what is a magical number so magical number means that the adjacent digits should have a difference equal to 1. Okay so summarizing everything in Hindi is less than equal to what digit okay digit name okay so now let us see so let us see some examples like we have one two so we can see that the a

In [35]:
def get_video_title(url):
    try:
        yt = YouTube(url)
        return yt.title
    except Exception as e:
        print("Title not found")
        return None

In [37]:
title = get_video_title("https://www.youtube.com/watch?v=GGJ5k8U5ccM")

Title not found


In [64]:
openai = OpenAI()
system_prompt = "You are an assistant that analyzes the contents of a transcript\
    and provide a title and short summary in mentioned langugae, ignoring text that is unwanted or navigation related.\
        respond in markdown."

In [70]:
def get_user_prompt(fulltext):
    user_prompt = f"You are looking at transcript of a youtube video and it is as follows\
        Please provide a title and short summary as paragraph with points in them for this video transcript in tamil in markdown.\n\n"
    user_prompt += full_text
    return user_prompt

In [71]:
print(get_user_prompt(full_text))

You are looking at transcript of a youtube video and it is as follows        Please provide a title and short summary as paragraph with points in them for this video transcript in tamil in markdown.

foreign [Music] foreign [Music] ERS today we will be discussing the problem of the day which is jumping numbers so I found this problem absolutely very interesting so let us just quickly start hi vedant hi yes hi Shiva CR Lee fgmg gives me a present okay fair enough so let's start with this problem so I think you have already read about this problem now in this problem you are asked so let us first understand that you are given a value X and you need to find a value which is a magical number okay now the question is that you need to find the nearest magical number to x nearest means less than equal okay so now the question is what is a magical number so magical number means that the adjacent digits should have a difference equal to 1. Okay so summarizing everything in Hindi is less than eq

In [72]:
message = [
    {"role" : "system", "content" : system_prompt},
    {"role" : "user", "content" : get_user_prompt(full_text)}
]

In [73]:
def summarize():
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = message
    )
    return response.choices[0].message.content

In [74]:
summarize()

"# குதி எண்கள் (Jumping Numbers)\n\nஇந்த வீடியோவில் 'குதி எண்கள்' என்ற சிக்கலுக்கு அறிவுரை வழங்கப்படுகிறது. இங்கு பரந்த விளக்கங்களுடன், முக்கிய புள்ளிகள் கீழே வழங்கப்பட்டுள்ளன:\n\n- **சிக்கல் விளக்கம்**: குதிக்கின்ற எண்கள் ஒரு பொருள், அதில் அடுத்த அடுக்கு எண்களுக்கு இடையே மாறுபாடு 1 ஆக இருக்க வேண்டும்.\n- **உதாரணங்கள்**: 89 மற்றும் 87 ஆகிய எண்கள் குதிக்கின்ற எண்கள், ஆனால் 796 மாதிரியான எண்கள் அல்ல.\n- **முதல் அணுகுமுறை**: வழங்கப்படும் எண்ணிற்குள் (X) அனைத்துக் குதிக்கின்ற எண்களை சரிபார்க்கும் முறையே குதிக்கின்ற எண்ணை கண்டறியலாம்.\n- **காலக்கNevertheless**: X வரம்பு 10^9 ஆக இருக்க முடியும்; எனவே, எல்லா எண்ணையும் சரிபார்க்கும் முறையின் காலநிலை நேரத்தின் மூலமாக அதிகமாக ஆகும்.\n- **அடுத்த அணுகுமுறை**: ஒரு தரவுப் மரம் உருவாக்கி, கணக்கிடும் முறையை மேம்படுத்துவது மற்றும் பிரத்தியேகமான எண்ணைக் கணக்கிடுவதை பேசுகிறது.\n- **கோடைமை (Recursive Approach)**: குறிப்பிட்ட முறைப்படி எண்ணுகளை உருவாக்கி, அதன் அடிப்படையில் மனுஷ்யத்தை வடிவமைப்பது.\n\nஇது கோடுகளை உருவாக்க, நிலைத்த சிக்கல்களை தீர்க்க பயன்படுத

In [75]:
def display_summary():
    summary = summarize()
    display(Markdown(summary))

In [76]:
display_summary()

# கூட்டிய எண்கள் - ஒரு மந்திர எண் தேடல்

இந்த காணொளியில் 'கூட்டிய எண்கள்' என்ற சிக்கலுக்கு உரித்தான விவாதம் இடம்பெற்றது. அதிலுள்ள முக்கிய அம்சங்கள்:

- **மந்திர எண்:** அடுத்த பட்டியலிலுள்ள எண்களின் இடைவெளி 1 ஆக இருக்க வேண்டும்.
- **குழப்பம்:** காணொளியில், எண்களை கழித்து எண்ண்பவர்களுக்கான தொடக்கத்திற்காக மதிப்பீடு செய்ய வேண்டிய மந்திர எண்ணை கண்டுபிடிக்க விரும்புகிறோம்.
- **எண் எடுத்துக்காட்டுகள்:** 89 மற்றும் 87 ஆகிய எண்கள் மந்திர எண்களாகவும், 796 எண் மந்திர எண் இதழுக்கு உட்பட்டதல்ல.
- **முதல் அணுகுமுறை:** சாதாரண ஒழுங்கில் ஒரு முதல் எண்ணிலிருந்து x வரையிலான அனைத்து எண்களையும் பரிசோதிக்கிறது.
- **இணைப்பு:** ஒரு எண் மந்திர எண் என காணப்படுதா என்பதை உறுதிப்படுத்தும் முறை.
- **காலத் தாமதம்:** முதன்மை கூறுகளின் எண்ணிக்கையை சரிபார்க்கவேண்டிய ஒருபாடு பெற்றால், இது 10^9 என்பதால் செயலதிகாரம் அதிகம் ஆகும்.
- **மீண்டும் அணுகுமுறை:** ஓரளவுக்கு கணினியியல் புத்திசாலித்தனம் மூலம் முக்கிய எண்களை வைத்துக்கொண்டு தீர்வுகளை உருவாக்குதல்.